In [0]:
with cte_data (strt, tgts, runs) as
(
select split_part(value, ': ', 1)
     , split(split_part(value, ': ', 2), ' ')
     , runs
--from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_11_input.txt', format => 'text', wholeText => false) as t (value)
/*from values ('aaa: you hhh')
            , ('you: bbb ccc')
            , ('bbb: ddd eee')
            , ('ccc: ddd eee fff')
            , ('ddd: ggg')
            , ('eee: out')
            , ('fff: out')
            , ('ggg: out')
            , ('hhh: ccc fff iii')
            , ('iii: out')  as t (value) -- part1 example */
  from values ('svr: aaa bbb')
            , ('aaa: fft')
            , ('fft: ccc')
            , ('bbb: tty')
            , ('tty: ccc')
            , ('ccc: ddd eee')
            , ('ddd: hub')
            , ('hub: fff')
            , ('eee: dac')
            , ('dac: fff')
            , ('fff: ggg hhh')
            , ('ggg: out')
            , ('hhh: out') as t (value) -- part2 example
  join values ( array( map('root', 'you', 'end', 'out')
                     , map('root', 'svr', 'end', 'fft')
                     , map('root', 'fft', 'end', 'dac')
                     , map('root', 'dac', 'end', 'out')
                     , map('root', 'svr', 'end', 'dac')
                     , map('root', 'dac', 'end', 'fft')
                     , map('root', 'fft', 'end', 'out')
                     ) ) as u (runs)
)
  ,  cte_prep (runs, graph) as
(
  select any_value(runs)
       , map_from_entries(array_agg(struct(strt, tgts)))
    from cte_data
)
  ,  cte_tree (run, cnt) as
(
select run
     , aggregate( sequence(1, 2500)
                , struct( array( struct( run.root        as node
                                       , 0               as idx
                                       , 0l              as sum
                                       , array(run.root) as path
                                       ) ) as stack
                        , map()::map<string, bigint> as scores )
                , (acc, x) -> aggregate( array()::array<int> -- fake aggregate to alias an expression
                                       , struct( element_at(acc.stack, -1) as ls
                                               , case
                                                   when size(acc.stack) > 1
                                                   then slice(acc.stack, 1, size(acc.stack) - 1)
                                                   else array()::array<struct<node:string,idx:int,sum:bigint,path:array<string>>>
                                                 end as sk_sliced)
                                       , (agg, y) -> null
                                       , agg -> case
                                                    -- recursion is fininshed: does not process anything
                                                  when size(acc.stack) = 0
                                                  then acc
                                                    -- reached end node: pop, +1 to parent
                                                  when agg.ls.node = run.end
                                                  then struct( transform( agg.sk_sliced
                                                                        , (f, i) -> case i when size(acc.stack) - 2 then struct(f.node, f.idx, f.sum + 1 as sum, f.path) else f end) as stack
                                                             , acc.scores )
                                                    -- cached: use score, pop
                                                  when acc.scores[agg.ls.node] is not null
                                                  then struct( transform( agg.sk_sliced
                                                                        , (f, i) -> case i when size(acc.stack) - 2 then struct(f.node, f.idx, f.sum + acc.scores[agg.ls.node] as sum, f.path) else f end) as stack
                                                             , acc.scores )
                                                    -- leaf: pop and cache 0 score
                                                  when graph[agg.ls.node] is null
                                                  then struct( agg.sk_sliced as stack
                                                             , map_concat(acc.scores, map(agg.ls.node, 0l)) as scores )
                                                    -- neighbors skip
                                                  when agg.ls.idx < size(graph[agg.ls.node])
                                                   and array_contains(agg.ls.path, graph[agg.ls.node][agg.ls.idx])
                                                  then struct( array_append( agg.sk_sliced
                                                                           , struct( agg.ls.node
                                                                                   , agg.ls.idx + 1 as idx
                                                                                   , agg.ls.sum
                                                                                   , agg.ls.path ) ) as stack
                                                             , acc.scores )
                                                    -- neighbor cached
                                                  when agg.ls.idx < size(graph[agg.ls.node])
                                                   and acc.scores[graph[agg.ls.node][agg.ls.idx]] is not null
                                                  then struct( array_append( agg.sk_sliced
                                                                           , struct( agg.ls.node
                                                                                   , agg.ls.idx + 1 as idx
                                                                                   , agg.ls.sum + acc.scores[graph[agg.ls.node][agg.ls.idx]] as sum
                                                                                   , agg.ls.path ) ) as stack
                                                             , acc.scores )
                                                    -- push neighbor
                                                  when agg.ls.idx < size(graph[agg.ls.node])
                                                  then struct( array_append( array_append( agg.sk_sliced
                                                                                         , struct( agg.ls.node
                                                                                                 , agg.ls.idx + 1 as idx
                                                                                                 , agg.ls.sum
                                                                                                 , agg.ls.path ) )
                                                                           , struct( graph[agg.ls.node][agg.ls.idx] as node
                                                                                   , 0  as idx
                                                                                   , 0l as sum
                                                                                   , array_append(agg.ls.path, graph[agg.ls.node][agg.ls.idx]) as path )
                                                                           ) as stack
                                                             , acc.scores )
                                                    -- cache, pop, add to parent
                                                  else struct( transform( agg.sk_sliced
                                                                        , (f, i) -> case i when size(acc.stack) - 2 then struct(f.node, f.idx, f.sum + agg.ls.sum as sum, f.path) else f end) as stack
                                                             , map_concat(acc.scores, map(agg.ls.node, agg.ls.sum)) as scores )
                                                end
                                       )
                , acc -> acc.scores[run.root]
                ) as cnt
  from cte_prep
  join lateral explode(runs) as t (run)
)
select sum(cnt) filter(where run.root = 'you' and run.end = 'out') as part1
     , sum(cnt) filter(where run.root = 'svr' and run.end = 'fft')
     * sum(cnt) filter(where run.root = 'fft' and run.end = 'dac')
     * sum(cnt) filter(where run.root = 'dac' and run.end = 'out')
     + sum(cnt) filter(where run.root = 'svr' and run.end = 'dac')
     * sum(cnt) filter(where run.root = 'dac' and run.end = 'fft')
     * sum(cnt) filter(where run.root = 'fft' and run.end = 'out') as part2
  from cte_tree;



